# Object Detection

The method used in this object detection algorithm is EfficientDet-Lite, a pretrained object detector. This is trained on COCO 2017 eval set and achieves mAP 41.5. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from scipy.spatial import distance as dist
import tensorflow_hub as hub
import cv2
import glob
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite4/detection/1") #link with image classifier
#link is subject to change due to updates.
#If this cell throws an error, increase the number in front of "lite" by 1. 
#Ex: lite4 -> lite5

In [ ]:
labels = pd.read_csv('/content/gdrive/MyDrive/GTVD-(DS440-Team1)/Data/labels.csv', sep=';', index_col='ID') # Download the label file from github
labels = labels['SUPER CATEGORY']

In [ ]:
def check_animal_person(pred_labels, pred_scores):
  prob_greater_0_5 = [index for index,value in enumerate(pred_scores) if value > 0.5]
  final_label = []
  for i in prob_greater_0_5:
    final_label.append(pred_labels[i])
  
  return set(['person', 'animal']) <= set(final_label)

In [ ]:
def detect_objects(input_filepath, output_filepath, width=1028, height=1028, show=False):
  # load data and preprocessing
  img = cv2.imread(input_filepath)
  if width is None or height == None:
    inp = img
  else:
    inp = cv2.resize(img, (width, height))
  rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
  rgb_tensor = tf.convert_to_tensor(rgb, dtype = tf.uint8)
  rgb_tensor = tf.expand_dims(rgb_tensor, 0)

  # run model
  boxes, scores, classes, num_detection = detector(rgb_tensor)

  # post processing
  pred_labels = classes.numpy().astype('int')[0] 
  pred_labels = [labels[i] for i in pred_labels]
  pred_boxes = boxes.numpy()[0].astype('int')
  pred_scores = scores.numpy()[0]
  
  detect_objects.checker = check_animal_person(pred_labels, pred_scores)
  
  if detect_objects.checker:
    # draw boxes on the figure
    output_image = rgb
    detect_objects.fileID = output_filepath.split('/')[-1]
    human_coordinates = []
    animal_coordinates = []
    for score, (ymin, xmin, ymax, xmax), label in zip(pred_scores, pred_boxes, pred_labels):
      if score < 0.25:
        continue

      if score > 0.25 and label == 'person':
        human_coordinates.append((ymin, xmin, ymax, xmax))
      
      if score > 0.25 and label == 'animal':
        animal_coordinates.append((ymin, xmin, ymax, xmax))

      # draw box    
      # i am thinking just 
      # score_txt = f'{100 * round(score)}%'
      score_txt = f'{round(100 * score)}%'
      output_image = cv2.rectangle(output_image, (xmin, ymax),(xmax, ymin),(0,255,0),2)
  

      # put text
      font = cv2.FONT_HERSHEY_SIMPLEX
      output_image = cv2.putText(output_image, label, (xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
      # output_image = cv2.putText(output_image, score_txt, (xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
      
    # save figure to the specified path
    ### if the predicted labels do not contain human and animals at the same time 
    ### we will not save the image to the output file
    cv2_output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_filepath, cv2_output_image)
    # print(human_coordinates)
    # print(animal_coordinates)
    detect_objects.human_target, detect_objects.animal_target, detect_objects.min_dis, detect_objects.human_width, detect_objects.human_length, detect_objects.animal_width, detect_objects.animal_length, detect_objects.human_area, detect_objects.animal_area, detect_objects.area_ratio = feature_creation(human_coordinates, animal_coordinates)
    print(detect_objects.fileID)
    print(detect_objects.min_dis)
  
    # show figure if "show" is true
    if show:
      plt.figure(figsize=(10,10))
      plt.imshow(output_image)

In [ ]:
# You could create feature you want here. 
def feature_creation(human_coordinates, animal_coordinates):
  min_dis = 10000
  human_index = 0
  animal_index = 0
  for i in range(len(human_coordinates)):
    for j in range(len(animal_coordinates)):
      human_mid_x = (human_coordinates[i][1] + human_coordinates[i][3])/2
      human_mid_y = (human_coordinates[i][0] + human_coordinates[i][2])/2
      animal_mid_x = (animal_coordinates[j][1] + animal_coordinates[j][3])/2
      animal_mid_y = (animal_coordinates[j][0] + animal_coordinates[j][2])/2
      distance = ((human_mid_x - animal_mid_x)**2 + (human_mid_y - animal_mid_y)**2)**0.5
      if distance < min_dis:
        min_dis = distance
        human_index = i
        animal_index = j
      else:
        continue
  # print(human_index)
  # print(animal_index)
  human_target = human_coordinates[human_index]
  animal_target = animal_coordinates[animal_index]
  human_width = abs(human_target[3]-human_target[1])
  human_length = abs(human_target[2]-human_target[0])
  animal_width = abs(animal_target[3]-animal_target[1])
  animal_length = abs(animal_target[2]-animal_target[0])
  human_area = human_width * human_length
  animal_area = animal_width * animal_length

  area_list = [human_area, animal_area]
  area_ratio = max(area_list) / min(area_list)

  return human_target, animal_target, min_dis, human_width, human_length, animal_width, animal_length, human_area, animal_area, area_ratio

In [ ]:
# os.chdir('/content/gdrive/My Drive/Pig_new')
# !ls

In [ ]:
# !pip install natsort

In [ ]:
# from natsort import natsorted

In [ ]:
# os.listdir casues some randomness when reading the filenames
# This function helps make sure it will follow the order exactly in the drive 
# Then, the table can be directly used along with the images when training
# import re
# numbers = re.compile(r'(\d+)')
# def numericalSort(value):
#     parts = numbers.split(value)
#     parts[1::2] = map(int, parts[1::2])
#     return parts

In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
input_folder = "/content/gdrive/MyDrive/GTVD-(DS440-Team1)/Data/Data_collection1/Benchmark_validation/Non_violation" # Set to the location of input file
filenames = sorted_alphanumeric(os.listdir(input_folder))
extensions = ['jpg', 'bmp', 'png', 'JPG', 'JPEG', 'jpeg']
filenames = [filename for filename in filenames] #if image type is not .jpg, replace with appropirate photo type
# filenames = sorted(filenames, key=numericalSort)
# filenames = natsorted(filenames)
len(filenames)

45

In [ ]:
output_folder = "/content/gdrive/MyDrive/Multi_input_data/Non-Violation_validation_lion" # Create a destination folder for images with objects detected
os.makedirs(output_folder, exist_ok=True)

In [ ]:
fileID = []
min_dis = []
human_width = []
human_length = []
human_area = []
animal_width = []
animal_length = []
animal_area = []
area_ratio = []
human_target = []
animal_target = []

for index, filename in enumerate(filenames, 1):
  print("Processing {} / {} - {}".format(index, len(filenames), filename))
  detect_objects(
    input_filepath = os.path.join(input_folder, filename),
    output_filepath = os.path.join(output_folder, filename),
    show=False, # you can turn this on if needed  
  )
  if detect_objects.checker:
    fileID.append(detect_objects.fileID)
    min_dis.append(detect_objects.min_dis)
    human_width.append(detect_objects.human_width)
    human_length.append(detect_objects.human_length)
    animal_width.append(detect_objects.animal_width)
    animal_length.append(detect_objects.animal_length)
    human_area.append(detect_objects.human_area)
    animal_area.append(detect_objects.animal_area)
    area_ratio.append(detect_objects.area_ratio)
    human_target.append(detect_objects.human_target)
    animal_target.append(detect_objects.animal_target)


Processing 1 / 45 - IMG_1736.JPG
Processing 2 / 45 - IMG_1737.JPG
Processing 3 / 45 - IMG_1738.JPG
Processing 4 / 45 - IMG_1739.JPG
Processing 5 / 45 - IMG_1741.JPG
Processing 6 / 45 - IMG_1742.JPG
IMG_1742.JPG
785.4457333259886
Processing 7 / 45 - IMG_1743.JPG
IMG_1743.JPG
780.435295203901
Processing 8 / 45 - IMG_1744.JPG
Processing 9 / 45 - IMG_1815.JPG
Processing 10 / 45 - IMG_1816.JPG
Processing 11 / 45 - IMG_1817.JPG
Processing 12 / 45 - IMG_1828.JPG
Processing 13 / 45 - IMG_1833.JPG
Processing 14 / 45 - IMG_1843.JPG
Processing 15 / 45 - IMG_1844.JPG
IMG_1844.JPG
591.3281660803923
Processing 16 / 45 - IMG_1845.JPG
IMG_1845.JPG
539.0326984515874
Processing 17 / 45 - IMG_1882.JPG
Processing 18 / 45 - IMG_1883.JPG
Processing 19 / 45 - IMG_1884.JPG
Processing 20 / 45 - IMG_2076.JPG
Processing 21 / 45 - IMG_2080.JPG
Processing 22 / 45 - IMG_2082.JPG
Processing 23 / 45 - IMG_2083.JPG
Processing 24 / 45 - IMG_2109.JPG
Processing 25 / 45 - IMG_2592.JPG
Processing 26 / 45 - IMG_2593.JPG
Pr

In [ ]:
df = pd.DataFrame(list(zip(fileID, min_dis, human_target, human_width, human_length, human_area, animal_target, animal_width, animal_length, animal_area, area_ratio)),
               columns =['File Name', 'Closest Distance', 'Human Target', 'Human Width', 'Human Length', 
                         'Human Area', 'Animal Taget', 'Animal Width', 'Animal Length', 'Animal Area', 'Area Ratio'])
print(df.head())
print(df.shape)

      File Name  Closest Distance           Human Target  Human Width  \
0  IMG_1742.JPG        785.445733   (241, 91, 1016, 340)          249   
1  IMG_1743.JPG        780.435295   (237, 57, 1023, 321)          264   
2  IMG_1844.JPG        591.328166  (10, 326, 1023, 1019)          693   
3  IMG_1845.JPG        539.032698  (88, 358, 1009, 1026)          668   
4  IMG_8563.JPG        449.651254   (393, 635, 490, 694)           59   

   Human Length  Human Area           Animal Taget  Animal Width  \
0           775      192975  (235, 829, 346, 1020)           191   
1           786      207504   (241, 802, 358, 990)           188   
2          1013      702009    (600, 11, 709, 184)           173   
3           921      615228    (600, 58, 714, 270)           212   
4            97        5723   (650, 255, 996, 598)           343   

   Animal Length  Animal Area  Area Ratio  
0            111        21201    9.102165  
1            117        21996    9.433715  
2            109    

In [ ]:
label = [0] * df.shape[0]
df['Label'] = label
df

,File Name,Closest Distance,Human Target,Human Width,Human Length,Human Area,Animal Taget,Animal Width,Animal Length,Animal Area,Area Ratio,Label
0,IMG_1742.JPG,785.445733,"(241, 91, 1016, 340)",249,775,192975,"(235, 829, 346, 1020)",191,111,21201,9.102165,0
1,IMG_1743.JPG,780.435295,"(237, 57, 1023, 321)",264,786,207504,"(241, 802, 358, 990)",188,117,21996,9.433715,0
2,IMG_1844.JPG,591.328166,"(10, 326, 1023, 1019)",693,1013,702009,"(600, 11, 709, 184)",173,109,18857,37.228032,0
3,IMG_1845.JPG,539.032698,"(88, 358, 1009, 1026)",668,921,615228,"(600, 58, 714, 270)",212,114,24168,25.456306,0
4,IMG_8563.JPG,449.651254,"(393, 635, 490, 694)",59,97,5723,"(650, 255, 996, 598)",343,346,118678,20.737026,0


In [ ]:
df.to_csv('non-violation_validation.csv')
!cp non-violation_validation.csv "/content/gdrive/MyDrive/Multi_input_data"